In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import time

from ssd_model import SSD300, SSD512
from ssd_utils import PriorUtil
from ssd_data import InputGenerator
from ssd_training import SSDLoss, SSDFocalLoss
from utils.model import load_weights
from utils.training import LearningRateDecay, Logger

### Data

#### Pascal VOC

In [ ]:
from data_voc import GTUtility
gt_util = GTUtility('data/VOC2007/')
gt_util_train, gt_util_val = gt_util.split(gt_util, split=0.8)
experiment = 'ssd512_voc'
#print(gt_util)

#### MS COCO

In [ ]:
from data_coco import GTUtility
gt_util = GTUtility('data/COCO/')
gt_util_train, gt_util_val = gt_util.split(gt_util, split=0.8)
experiment = 'ssd512_coco'
#print(gt_util)

### Model

In [ ]:
#model = SSD300(num_classes=gt_util.num_classes)
model = SSD512(num_classes=gt_util.num_classes)

prior_util = PriorUtil(model)

### Weights

In [ ]:
initial_epoch = 0

weights_path = '~/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
layer_list = [('block1_conv1', 'conv1_1'),
              ('block1_conv2', 'conv1_2'),
              ('block2_conv1', 'conv2_1'),
              ('block2_conv2', 'conv2_2'),
              ('block3_conv1', 'conv3_1'),
              ('block3_conv2', 'conv3_2'),
              ('block3_conv3', 'conv3_3'),
              ('block4_conv1', 'conv4_1'),
              ('block4_conv2', 'conv4_2'),
              ('block4_conv3', 'conv4_3'),
              ('block5_conv1', 'conv5_1'),
              ('block5_conv2', 'conv5_2'),
              ('block5_conv3', 'conv5_3')]
#load_weights(model, weights_path, layer_list)

#weights_path = './models/ssd512_voc_weights_fixed.hdf5'
#weights_path = './checkpoints/201709251316_ssd512_coco/weights.040.h5'; initial_epoch = 41
weights_path = './checkpoints/201709081909_ssd512_voc/weights.034.h5'; initial_epoch = 35

model.load_weights(weights_path, by_name=True)

freeze = ['conv1_1', 'conv1_2',
          'conv2_1', 'conv2_2',
          'conv3_1', 'conv3_2', 'conv3_3',
          #'conv4_1', 'conv4_2', 'conv4_3',
          #'conv5_1', 'conv5_2', 'conv5_3',
         ]

### Training

In [ ]:
epochs = 100
batch_size = 24

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size, augmentation=True)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size, model.image_size, augmentation=True)

# freeze layers
for l in model.layers:
    l.trainable = not l.name in freeze

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

#optim = keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0, nesterov=True)
optim = keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# weight decay
regularizer = keras.regularizers.l2(5e-4) # None if disabled
#regularizer = None
for l in model.layers:
    if l.__class__.__name__.startswith('Conv'):
        l.kernel_regularizer = regularizer

loss = SSDLoss(alpha=1.0, neg_pos_ratio=3.0)
#loss = SSDFocalLoss()

model.compile(optimizer=optim, loss=loss.compute, metrics=loss.metrics)
        
history = model.fit_generator(
        gen_train.generate(), 
        steps_per_epoch=gen_train.num_batches*3,
        epochs=epochs, 
        verbose=1, 
        callbacks=[
            keras.callbacks.ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, save_weights_only=True),
            Logger(checkdir),
            #LearningRateDecay(methode='linear', base_lr=1e-3, n_desired=40000, desired=0.1, bias=0.0, minimum=0.1)
        ], 
        validation_data=gen_val.generate(), 
        validation_steps=gen_val.num_batches, 
        class_weight=None, 
        #max_queue_size=10, 
        workers=1, 
        #use_multiprocessing=False, 
        initial_epoch=initial_epoch)